In [ ]:
import xarray as xr
from cmocean import cm as cmo
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [ ]:
ds = xr.open_dataset('sg579_EUREC4A_1.0m_up_and_down_profile.nc')
ds

In [ ]:
profile_time = ds.ctd_time.mean(axis=1).data # average time per profile
ds = ds.assign_coords(profile_time=("profile", profile_time)) # assign this as a new coordinate called profile_time
ds.profile_time.attrs =  {'comment': 'Mean time of CTD profile', # add essential metadata, or xarray won't plot it
 'standard_name': 'time',
 'axis': 'profile'}

In [ ]:
def prepare_for_plotting(dataset, variable):
    """Prepare variable for plotting by:
    1. Removing outliers more than 3 std dev from the mean
    2. Interpolating over nans
    """
    data = ds[variable].data
    # Copy some of the cleaning functionality from GliderTools and use it here
    # https://github.com/GliderToolsCommunity/GliderTools/blob/master/glidertools/cleaning.py
    # e.g. remove data more than 2 standard deviations from the mean
    
    from numpy import array, nan, nanmean, nanstd
    arr = data

    mean = nanmean(arr)
    std = nanstd(arr)
    
    multiplier = 3
    ll = mean - std * multiplier
    ul = mean + std * multiplier

    mask = (arr < ll) | (arr > ul)
    arr[mask] = nan
    
    ds[variable].data = data
    return dataset

In [ ]:
# define a basic plotting function for the profiles
def plotter(dataset, variable,colourmap,title):
    """TODO: doc string here"""
    fig, ax = plt.subplots()
    #xr.plot.pcolormesh(darray = ds, x = 'profile_time', y = 'depth', z = variable, cmap = colourmap)
    dataset = prepare_for_plotting(dataset, variable)
    dataset[variable].T.plot(yincrease=False, y="depth", x="profile_time", cmap = colourmap)
    ax.set_title(str(title))
    plt.tight_layout()
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%d/%m')) # sets x tick format
    return fig, ax

In [ ]:
a = list(ds.keys()) # list data variables in ds

In [ ]:
# list of variables from old matlab script that may come from glider
glider_variables = ('temp','temperature','conductivity','salinity','sigma_theta','dissolved_oxygen_sat',
'eng_aa4330_O2','eng_aa4330F_O2','eng_aa1_O2','eng_aa807_O2','eng_aa4831_O2','contopt_instrument_dissolved_oxygen','contopt_dissolved_oxygen',
'eng_aa4330_Temp','eng_aa4330F_Temp','eng_aa1_Temp','eng_aa807_Temp','eng_aa4831_Temp','eng_contopt_Temp', 'eng_contopt_pO2',
'eng_wl_sig1','eng_wl1_sig1','eng_wl_Chlsig1','eng_wl1_Chlsig1','eng_wl_Cdomsig1','eng_wl1_Cdomsig1',
'eng_phfl_raw_data','eng_phfl_pHcalib','eng_phfl_Temp',
'eng_ph_pH','eng_ph_pCO2','eng_ph_temp_pH',
'eng_qsp144_PARuV','eng_qsp1_PARuV','eng_qsp_PARuV','eng_qsp_PARuV','eng_qsp1_PARuv',
'eng_wlbbfl2vmt_wl600sig','eng_wlbbfl2vmt_wl650sig','eng_wlbbfl2vmt_Cdomsig','eng_wlbbfl2vmt_Chlsig',
'eng_wl1_sig1','eng_wl1_Chlsig1','eng_wl1_Cdomsig1',
'eng_wl836_sig1','eng_wl836_Chlsig1','eng_wl836_Cdomsig1',
'eng_wlbb2fl_BB1sig','eng_wlbb2fl_BB2sig','eng_wlbb2fl_FL1sig',
'eng_wlbbfl2_BB1sig','eng_wlbbfl2_FL1sig','eng_wlbbfl2_FL2sig',
'eng_wlseaowl_FL1sig','eng_wlseaowl_BB1sig','eng_wlseaowl_FDOMsig')

In [ ]:
#create dictionary to match each variable in glider_variables to a colourmap
dict = {'temp':cmo.thermal,'temperature':cmo.thermal,'conductivity':cmo.haline,'salinity':cmo.haline,'sigma_theta':cmo.dense,'dissolved_oxygen_sat':cmo.oxy,
'eng_aa4330_O2':cmo.oxy,'eng_aa4330F_O2':cmo.oxy,'eng_aa1_O2':cmo.oxy,'eng_aa807_O2':cmo.oxy,'eng_aa4831_O2':cmo.oxy,
'contopt_instrument_dissolved_oxygen':cmo.oxy,'contopt_dissolved_oxygen':cmo.oxy,
'eng_aa4330_Temp':cmo.thermal,'eng_aa4330F_Temp':cmo.thermal,'eng_aa1_Temp':cmo.thermal,'eng_aa807_Temp':cmo.thermal,'eng_aa4831_Temp':cmo.thermal,
'eng_contopt_Temp':cmo.thermal, 'eng_contopt_pO2':cmo.oxy, 'eng_wl_sig1': cmo.gray,'eng_wl1_sig1': cmo.gray,'eng_wl_Chlsig1': cmo.algae,
'eng_wl1_Chlsig1': cmo.algae,'eng_wl_Cdomsig1': cmo.gray,'eng_wl1_Cdomsig1': cmo.gray,
'eng_phfl_raw_data': cmo.gray,'eng_phfl_pHcalib': cmo.gray,'eng_phfl_Temp': cmo.gray,
'eng_ph_pH': cmo.gray,'eng_ph_pCO2': cmo.gray,'eng_ph_temp_pH': cmo.gray,
'eng_qsp144_PARuV': cmo.gray,'eng_qsp1_PARuV': cmo.gray,'eng_qsp_PARuV': cmo.gray,'eng_qsp_PARuV': cmo.gray,'eng_qsp1_PARuv': cmo.gray,
'eng_wlbbfl2vmt_wl600sig': cmo.gray,'eng_wlbbfl2vmt_wl650sig': cmo.gray,'eng_wlbbfl2vmt_Cdomsig': cmo.gray,'eng_wlbbfl2vmt_Chlsig': cmo.gray,
'eng_wl1_sig1': cmo.gray,'eng_wl1_Chlsig1': cmo.gray,'eng_wl1_Cdomsig1': cmo.gray,
'eng_wl836_sig1': cmo.gray,'eng_wl836_Chlsig1': cmo.gray,'eng_wl836_Cdomsig1': cmo.gray,
'eng_wlbb2fl_BB1sig': cmo.gray,'eng_wlbb2fl_BB2sig': cmo.gray,'eng_wlbb2fl_FL1sig': cmo.gray,
'eng_wlbbfl2_BB1sig': cmo.gray,'eng_wlbbfl2_FL1sig': cmo.gray,'eng_wlbbfl2_FL2sig': cmo.gray,
'eng_wlseaowl_FL1sig': cmo.gray,'eng_wlseaowl_BB1sig': cmo.gray,'eng_wlseaowl_FDOMsig': cmo.gray}

#variables assigned to cmo.gray are yet to be assigned to the appropriate colormap

In [ ]:
to_plot  = list(set(a).intersection(glider_variables)) # find elements in glider_variables relevant to this dataset

In [ ]:
for i in range(len(to_plot)):
    plotter(ds, to_plot[i], dict[to_plot[i]], to_plot[i])